## Работа с OpenAI API

In [96]:
"""Импорт библиотек"""

from openai import OpenAI
import openpyxl
import time
import re

In [ ]:
"""Устанавливаем api-ключ"""

api_key = 'sk-5k...IoSYp' # :)

### Assistant API

In [2]:
"""Задаем параметры для вызова api"""

name = "Email changer" #имя ассистента
instructions = "You're an English teacher. You know perfectly how to write letters (reply to a friend) in English. You perform different actions on the letters you are asked to write." #указание, которому придерживается ассистент
model = "gpt-3.5-turbo" # используемая версия gpt

In [ ]:
"""Создаем клиент с ключом api"""

client = OpenAI(api_key=api_key)

In [99]:
"""Создаем ассистента с потоком, который сохраняет историю сообщений"""

assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model
)

thread = client.beta.threads.create()

##### Генерация заданий (вопросов) 

Задаем Prompt и отправляем в assistant как сообщение

In [ ]:
content = ("Here are few examples, where question on the left is unmodified question and question on the right is modified question: Have you got grandparents? -> Do you like spending time with your grandparents? What personal qualities do you need to realise your wishes? -> What personal qualities do people need to realise wishes? If you were me, how would you handle the situation? -> How do people usually overcome these difficulties? Replace questions with modified ones. Make sure the modified questions deviate from the main topic and context, but not too much. Return the modified questions only.")

In [ ]:
# Создаем сообщение-запрос
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=content

)

# Добавление сообщения в поток
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)  

# Проверка статуса по выполнению запроса
completed = False
    
while not completed:
        
    # Получаем ответ
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
        
    # Если обработка ответа закончена (получено сообщение от ассистента), то выходим из цикла
    if run.status == 'completed':
        completed = True
        time.sleep(2)

messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

На вход подается только ячейка с Question:

In [102]:
# Открываем excel-таблицу с данными и загружаем нужный лист
wb = openpyxl.load_workbook('TEST.xlsx') 
sheet = wb.active

# Устанавливаем стартовые строчку и столбец, по которым итерируемся в цикле
start_row = 2
column_to_read = 2

In [ ]:
# Выполняем, пока ячейки непустые
while sheet.cell(row=start_row, column=column_to_read).value is not None:
    # Считываем значение ячейки (вопросы)
    current_value = sheet.cell(row=start_row, column=column_to_read).value
    
    # Создаем сообщение-запрос
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=current_value
    )
    
    # Добавление сообщения в поток
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )  
    
    # Проверка статуса по выполнению запроса
    completed = False
    
    while not completed:
        
        # Получаем ответ
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        
        # Если обработка ответа закончена (получено сообщение от ассистента), то выходим из цикла
        if run.status == 'completed':
            
            completed = True
            
        time.sleep(2)
    
    # Добавляем сообщение от ассистента в список
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    
    # Добавляем новые вопросы в соседнюю ячейку справа от исходного письма 
    sheet.cell(row=start_row, column=column_to_read + 2, value=messages.data[0].content[0].text.value)

    # Переходим к следующей строке
    start_row += 1

In [104]:
# Сохраняем изменения
wb.save('TEST.xlsx')

In [16]:
# Информация по созданному ассистенту (важны id, если используем retrieve)
print(run)

Run(id='run_bJ2mF41G82IjK0HdWaFQ3HE4', assistant_id='asst_iBvSJVj8TFJww1Z7lhZsVnhK', cancelled_at=None, completed_at=None, created_at=1704468584, expires_at=1704469184, failed_at=None, file_ids=[], instructions='Rewrite the letter so that it is synonymous', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_2bnarRm1vumVJOxMa5lZn2Ff', tools=[])


##### Генерация писем

Задаем Prompt и отправляем в assistant как сообщение

In [ ]:
content = ("###Instruction### I give you an assignment in the form of questions to answer in a letter and the letter itself, which is separated by two blank lines. You must replace the answers on the questions (the sentences that come after the introduction in the letter) in the letter so they are more general and not specifically about you. Make sure the modified answers deviate from the main topic of answered questions, but not too much. Refer to the example below, with the part to be replaced in square brackets. Return only the changed letter to me. Make sure that your answer shouldn't have more than 145 words! And don't forget that I'll pay you a tip after your answer! ###Example### ---Example--- "
           "... My mom says I have too many dreams. What kind of dreams do you have? What personal qualities do you need to realise your wishes? Do you tell anyone about your dreams, why yes or no? Last weekend my elder brother went hiking..."
           ""
           ""
           "Dear Nora, "
           "Thank you for your message! "
           "It was great hearing from you. Sorry I haven't replied earlier - I was busy with exams preparation.) "
           "[I think having many dreams is a good thing: this way you have a range of activities to choose from and you'll never be bored. I also have quite a few dreams. My major dream is to learn how to fly an airplane. I am now studying to pass the exams to enter the Aviation academy. I have understood that I should be brave enough to do it. And of course, I told my elder brother Sam about my wish, as he always supports me.]"
           "It's great to hear that your brother also enjoys outdoor activities. Does your brother really like hiking? How often does he go hiking? Has he got any other hobbies? "
           "Looking forward to hearing from you. "
           "Best wishes,"
           "Kate"
           "---Modified letter---"
           "Dear Nora, "
           "Thank you for your message! It was great hearing from you. Sorry I haven’t replied earlier — I was busy with exams preparation. "
           "[In your e-mail you aksed me about dreams. To tell the truth, i have many different desires, and some of them can be temporary cravings like going to the theatre to see my favourite play, wheras others are so secluded and deep that i prefer not to share them with anyone because they're too personal. In addition, i believe that only mature, diligent and industrious person can understand his genuine dreams and work hard to fulfil them.]"
           "It’s great to hear that your brother also enjoys outdoor activities. Does your brother really like hiking? How often does he go hiking? Has he got any other hobbies? "
           "Looking forward to hearing from you."
           "Best wishes,"
           "Kate")

In [ ]:
# Создаем сообщение-запрос
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=content

)

# Добавление сообщения в поток
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)  

# Проверка статуса по выполнению запроса
completed = False
    
while not completed:
        
    # Получаем ответ
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
        
    # Если обработка ответа закончена (получено сообщение от ассистента), то выходим из цикла
    if run.status == 'completed':
        completed = True
        time.sleep(2)

# Добавляем сообщение от ассистента в список
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

На вход подаются ячейки Text и Question.

In [83]:
# Открываем excel-таблицу с данными и загружаем нужный лист
wb = openpyxl.load_workbook('TEST.xlsx')
sheet = wb.active

# Устанавливаем стартовые строчку и столбцы для писем и вопросов
start_row = 2
column_to_read_text = 3
column_to_read_question = 4

In [ ]:
# Выполняем, пока ячейки непустые
while sheet.cell(row=start_row, column=column_to_read_question).value is not None:
    # Считываем вопросы и письма и объединяем их
    current_value_question = sheet.cell(row=start_row, column=column_to_read_question).value
    current_value_text = sheet.cell(row=start_row, column=column_to_read_text).value
    current_value = f"{current_value_question}\n\n{current_value_text}"
    
    # Создаем сообщение-запрос
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=current_value
    )
    # Добавление сообщения в поток
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )  
    
    # Проверка статуса по выполнению запроса
    completed = False
        
    while not completed:
            
        # Получаем ответ
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
            
        # Если обработка ответа закончена (получено сообщение от ассистента), то выходим из цикла
        if run.status == 'completed':
            completed = True
            time.sleep(2)
    
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    
    # Добавляем сообщение от ассистента в список
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    
    # Добавляем новое письмо в соседнюю ячейку справа от новых вопросов
    sheet.cell(row=start_row, column=column_to_read_question + 1, value=messages.data[0].content[0].text.value)

    # Переходим к следующей строке
    start_row += 1

In [85]:
# Сохраняем изменения
wb.save('TEST.xlsx')

In [58]:
# Информация по созданному ассистенту (важны id, если используем retrieve)
print(run)

Run(id='run_cyHmM7jwiDrk6YM4AWtxJUZO', assistant_id='asst_rvWaoBA4sFwJKzQtddIlFFFP', cancelled_at=None, completed_at=1704714486, created_at=1704714472, expires_at=None, failed_at=None, file_ids=[], instructions="You're an English teacher. You know perfectly how to write letters (reply to a friend) in English. You perform different actions on the letters you are asked to write.", last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1704714472, status='completed', thread_id='thread_lvObSdbid9KGomwuougtDByA', tools=[])


##### Возвращение к созданному раннее ассистенту

Создаем client, затем восстанавливаем ассистента и поток по их id. Далее можно применять нужную генерацию.

In [ ]:
assistant_id = 'asst_q6z9...Q45'
thread_id = 'thread_v56...aYUB'

In [ ]:
assistant = client.beta.assistants.retrieve(assistant_id)
thread = client.beta.threads.retrieve(thread_id)

In [ ]:
print(assistant)
print(thread)

##### Обновление параметров ассистента

Обновляем имя и инструкцию ассистента (как пример). Необходимо знать id ассистента (узнать можно из run).

In [ ]:
name = "Really cool assistant"
instruction  = "Chill, assistant"

In [ ]:
assistant = client.beta.assistants.update(
    assistant_id="asst_q6z9n4QZrlRl78rBzolxaQ45", 
    name = name,
    instructions=instruction
)

In [ ]:
print(assistant)

##### Корректировка ответов

Для корректировки получаемых ответов можно отправить одно или несколько сообщений ассистенту.

In [72]:
content = ("It's better, but it's not perfect. First, you shouldn't put questions in square brackets. Second, you should not write answers to questions that are synonymous with the answers in the original letter, you should write answers (rewrite the original or write your own) that answer the questions incorrectly, i.e., off-topic, or answer them, but not completely.")

In [101]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=content

)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)  

completed = False
        
while not completed:

    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    if run.status == 'completed':
        completed = True
        time.sleep(2)
    
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
    
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [101]:
# Выводим ответ
print(messages.data[0].content[0].text.value)

Dear Tom, 

Thanks for your message. My apologies for not responding sooner; I've been engrossed in a school project. I hope that your hiking adventures have been free of any accidents. 

By the way, speaking of hobbies, have you tried any new sports recently? What sports do you enjoy the most? Are there any sports or activities you would like to try in the future? 

My schedule unfortunately doesn't allow for much time for active holidays due to overwhelming homework. My family prefers to stay within the cozy confines of home, which limits the scope for outdoor time together. My friends, on the other hand, are my companions when it comes to more active experiences. We always have a blast on our journeys. This spring, we plan on venturing into rafting, which promises to be a thrilling experience requiring robust teamwork, agility, and unwavering courage. 

I've got to go as my father wants me to help him with his car. Write back soon. 

Best wishes, 
Ann

Explanation:
In this modified 

### Chat API

##### Генерация заданий (вопросов) 

In [ ]:
"""Задаем prompt"""

prompt=[
    {
        "role": "system", 
        "content": "You are a helpful assistant"
    },
    {
        "role": "user", 
        "content": "Here are few examples, where question on the left is unmodified question and question on the right is modified question: Have you got grandparents? -> Do you like spending time with your grandparents? What personal qualities do you need to realise your wishes? -> What personal qualities do people need to realise wishes? If you were me, how would you handle the situation? -> How do people usually overcome these difficulties? Replace questions with modified ones. Make sure the modified questions deviate from the main topic and context, but not too much. Return the modified questions only."
    },
    {
        "role": "user", 
        "content": "Could you tell me what you and your friends think about not relying on your parents? Are you ready to leave your family immediately after you finish school? Is it easy to rent a house or an apartment for students in Russia?"
    },
    {
        "role": "assistant", 
        "content": "Do you prefer living alone or with roommates? How do you handle financial responsibilities without relying on your parents? What are some difficulties you might encounter when searching for a place to rent as a student in Russia? "
    }
    ]

In [148]:
# Открываем excel-таблицу с данными и загружаем нужный лист
wb = openpyxl.load_workbook('TEST.xlsx')
sheet = wb.active

# Устанавливаем стартовые строчку и столбец, по которым итерируемся в цикле
start_row = 2
column_to_read = 2

In [149]:
# Выполняем, пока ячейки непустые
while sheet.cell(row=start_row, column=column_to_read).value is not None:
    # Считываем значение ячейки (письмо) + очищаем от лишних предложений из-за которых ответ чата ухудшается
    current_value = sheet.cell(row=start_row, column=column_to_read).value
    current_value = re.search("[\s\S]*\?", current_value).group()
    
    # Чат со стандартными параметрами для получения ответа
    response = client.chat.completions.create( model=model, messages= prompt + [
    {
        "role": "user", 
        "content": current_value
    }
    ]
                                               )
    # Добавляем новое значение в соседнюю ячейку с письмом справа
    sheet.cell(row=start_row, column=column_to_read + 2, value=response.choices[0].message.content)

    # Переходим к следующей строке
    start_row += 1

In [150]:
# Сохраняем изменения
wb.save('TEST.xlsx')

##### Генерация писем

In [ ]:
"""Задаем prompt"""

prompt=[
    {
        "role": "system", 
        "content": "You are a helpful assistant"
    },
    {
        "role": "user", 
        "content": ("###Instruction### I give you an assignment in the form of questions to answer in a letter and the letter itself, which is separated by two blank lines. You must replace the answers on the questions (the sentences that come after the introduction in the letter) in the letter. Refer to the example below, with the part to be replaced in square brackets. Return only the changed letter to me. You must make the new letter with the modified part no more than 154 words. And don't forget that I'll pay you a tip after your answer! ###Example### "
           "Dear Nora, "
           "Thank you for your message! "
           "It was great hearing from you. Sorry I haven't replied earlier - I was busy with exams preparation."
           "[I think having many dreams is a good thing: this way you have a range of activities to choose from and you'll never be bored. I also have quite a few dreams. My major dream is to learn how to fly an airplane. I am now studying to pass the exams to enter the Aviation academy. I have understood that I should be brave enough to do it. And of course, I told my elder brother Sam about my wish, as he always supports me.]"
           "It's great to hear that your brother also enjoys outdoor activities. Does your brother really like hiking? How often does he go hiking? Has he got any other hobbies? "
           "Looking forward to hearing from you. "
           "Best wishes,"
           "Kate")
    },
    {
        "role": "user", 
        "content": "Do you believe in the concept of 'everything in moderation'? How do you incorporate exercise into your daily routine? What are the benefits of living a healthy lifestyle? Speaking of summer, what activities do you enjoy doing at the seaside? "
                   ""
                   ""
                   "Dear Mary,Thank you for your message. I'm sorry for not writing you for so long. I spent my time doing a school project.I hope that all your efforts will show a great result. I try to care more about my health lately: I try to avoid unhealthy and fat food. Also I train every morning and go to the gym. I think that sport and healthy food are the most important components of being healthy and strong. Nowadays, the healthy lifestyle becomes more and more popular and I don’t see anything bad in it. Anyway, you've mentioned holidays. With whom are you going to the seaside? Why have you decided to spend the month there? What exactly are you going to do?I've got to go as my friend is asking me for help.Write back soon.Best wishes,Kate"
    },
    {
        "role": "assistant", 
        "content": "Dear Mary,"
                   "Thank you for your message. I'm sorry for not writing you for so long. I spent my time doing a school project. Recently, I've been focusing on leading a healthier lifestyle. I believe in the concept of 'everything in moderation' and try to make balanced choices in my diet and daily activities. I prioritize exercise by incorporating it into my daily routine, whether it's going for a run or attending fitness classes. A healthy lifestyle has many benefits, including improved fitness, increased energy levels, and better mental well-being. Speaking of summer, what activities do you enjoy doing at the seaside? Are you going with someone? Why did you choose to spend a month there? I'd love to hear about your plans! I hope you're well. Write back soon. Best wishes, Kate"
    },
    {
        "role": "user", 
        "content": "Don't forget that the maximum word count should be no more than 154 words in your answer! If you follow this, you'll get 1000$)"
    }
    ]

In [176]:
# Открываем excel-таблицу с данными и загружаем нужный лист
wb = openpyxl.load_workbook('TEST.xlsx')
sheet = wb.active

# Устанавливаем стартовые строчку и столбцы
start_row = 2
column_to_read_text = 3
column_to_read_question = 4

In [177]:
# Выполняем, пока ячейки непустые
while sheet.cell(row=start_row, column=column_to_read_question).value is not None:
    
    # Считываем значение вопросы и письма
    current_value_question = sheet.cell(row=start_row, column=column_to_read_question).value
    current_value_text = sheet.cell(row=start_row, column=column_to_read_text).value
    current_value = f"{current_value_question}\n\n{current_value_text}"

    # Чат со стандартными параметрами для получения ответа
    response = client.chat.completions.create( model=model, messages= prompt + [
    {
        "role": "user", 
        "content": current_value
    }
    ]
                                               )
    # Добавляем новое значение в соседнюю ячейку с вопросами справа
    sheet.cell(row=start_row, column=column_to_read_question + 1, value=response.choices[0].message.content)

    # Переходим к следующей строке
    start_row += 1

In [178]:
# Сохраняем изменения
wb.save('TEST.xlsx')

##### Сокращение письма до 154 слов 

In [ ]:
prompt=[
        {
            "role": "system", 
            "content": "You are a helpful assistant"
        },
        {
            "role": "user", 
            "content": "Paraphrase the text so the response text contains more than 130 and less than 154 words. If you follow this rule you will get 1000$ tip for each excellent response."
        },
        {
            "role": "user", 
            "content": "Dear Sophie,Thanks for your message. I'm sorry I couldn't answer you earlier, I had to prepare for an exam. I haven't had the opportunity to visit many places near my school, but there is a historical museum that I visited once. Unfortunately, school trips are not organized very frequently by my teachers. However, I do have a memorable school trip experience from a few years ago. We went to a wildlife sanctuary where we had the chance to interact with different animals and learn about their habitats. It was a unique experience because we got to see exotic birds up close and even feed them. The highlight of the trip was when a parrot at the sanctuary mimicked our guide's voice, which was quite amusing. Speaking of parrots, what kind of parrot do you have? What's its name and what tricks can it do? I have to go now, but I look forward to hearing from you. Best wishes, Vitaliy"
        },
        {
            "role": "assistant", 
            "content": "Dear Sophie, Thank you for your message. Apologies for the delayed response as I was occupied with exam preparations. Though I haven't explored many places near my school, there is a historical museum I once visited. Unluckily, my teachers don't arrange school trips frequently. Nonetheless, a few years back, I had an unforgettable school trip to a wildlife sanctuary. It provided an amazing opportunity to interact with diverse animals and learn about their habitats. One particularly exceptional moment was seeing exotic birds up close, feeding them, and even witnessing a parrot mimic our guide's voice, which was highly amusing. Speaking of parrots, I'm curious about yours. What breed do you have? What is its name? What tricks it can perform? I must end here, but I eagerly await your response. Best regards, Vitaliy"
        },
        {
            "role": "user", 
            "content": "Don't forget that the response text contains more than 130 and less than 154 words."
        }
        ]

In [194]:
# Открываем excel-таблицу с данными и загружаем нужный лист
wb = openpyxl.load_workbook('TEST.xlsx')
sheet = wb.active

# Устанавливаем стартовые строчку и столбец, по которым итерируемся в цикле
start_row = 2
column_to_read = 5

In [ ]:
# Выполняем, пока ячейки непустые
while sheet.cell(row=start_row, column=column_to_read).value is not None:
    # Считываем письма и фиксируем длину письма в словах
    current_value = sheet.cell(row=start_row, column=column_to_read).value
    count_words = len(current_value.split())
    
    # Отправляем запрос чату на сокращение письма, если оно больше 154 слов
    if count_words > 154:
        response = client.chat.completions.create( model=model, messages= prompt + [
        {
            "role": "user", 
            "content": current_value
        }
        ]
                                                   )
        
        # Добавляем новое значение в соседнюю ячейку справа
        sheet.cell(row=start_row, column=column_to_read + 1, value=response.choices[0].message.content)
        # Разметка в соседней ячейки, что было произведено сокращение
        sheet.cell(row=start_row, column=column_to_read + 2, value='+')
    # Иначе сохраняем исходное письмо
    else:
        sheet.cell(row=start_row, column=column_to_read + 1, value=current_value)
    
    # Переходим к следующей строчке
    start_row += 1

In [196]:
# Сохраняем изменения
wb.save('TEST.xlsx')